In [1]:
import Text.Read
import Data.Maybe
import Data.Char
import Data.List.Split
import qualified Data.Vector as V
import qualified Data.Map as M

In [2]:
type Reg = Char
data Value = Value Int | Register Reg deriving Show
data Instruction = Send Value
                 | Set Reg Value
                 | Add Reg Value
                 | Mul Reg Value
                 | Mod Reg Value
                 | Receive Reg
                 | Jump Value Value deriving Show

In [3]:
readRegister = head . dropWhile isSpace
readValue s =
  let r = Register $ readRegister s
      v = Value <$> readMaybe s
    in fromMaybe r v

In [4]:
readLine :: String -> Instruction
readLine s = case i of
  "snd" -> Send (readValue $ args !! 0)
  "set" -> Set (readRegister $ args !! 0)
               (readValue $ args !! 1)
  "add" -> Add (readRegister $ args !! 0)
               (readValue $ args !! 1)
  "mul" -> Mul (readRegister $ args !! 0)
               (readValue $ args !! 1)
  "mod" -> Mod (readRegister $ args !! 0)
               (readValue $ args !! 1)
  "rcv" -> Receive (readRegister $ args !! 0)
  "jgz" -> Jump (readValue $ args !! 0)
                (readValue $ args !! 1)
  where i = take 3 s
        args = splitOn " " $ drop 4 s

In [10]:
type Machine = M.Map Reg Int
type Pc = Int

type Instructions = V.Vector Instruction
step :: Instructions -> (Machine, Pc) -> [Int]
                     -> Maybe (Machine, Pc, [Int], Maybe Int)
step ins (m, pc) c = 
  if (pc < 0 || pc >= V.length ins) then Nothing else
  case i of
  Send v -> Just $ (m, pc + 1, c, Just $ getValue v)
  Set a b -> Just $ (setValue a b, pc + 1, c, Nothing)
  Add a b -> Just $ (setValue a $ Value
                         $ (getValue b) +
                           (getValue $ Register a),
              pc + 1, c, Nothing)
  Mul a b -> Just $ (setValue a $ Value
                         $ (getValue b) *
                           (getValue $ Register a),
              pc + 1, c, Nothing)
  Mod a b -> Just $ (setValue a $ Value
                         $ (getValue $ Register a) `mod`
                           (getValue b),
              pc + 1, c, Nothing)
  Receive v -> if (null c)
               then Nothing
               else Just $ (setValue v $ Value $ head c, pc + 1, tail c, Nothing)
  Jump a b -> Just $ if (getValue a > 0) then (m, pc + getValue b, c, Nothing)
                                  else (m, pc + 1, c, Nothing)
  where i = ins V.! pc
        getValue (Value i) = i
        getValue (Register r) = fromMaybe 0 $ M.lookup r m
        setValue r v = M.insert r (getValue v) m

In [11]:
run :: Instructions -> Int -> [Int] -> [Int]
run ins i c = run' ins (M.singleton 'p' i, 0) c

run' ins mpc@(_, pc) ch =
  let r = (step ins mpc ch)
  in if isJust r then
    let (m', pc', ch', o) = fromJust r in
      if isJust o
        then fromJust o:(run' ins (m', pc') ch')
        else (run' ins (m', pc') ch')
    else []

In [7]:
ins = V.fromList [Set 'i' (Value 31),Set 'a' (Value 1),Mul 'p' (Value 17),Jump (Register 'p') (Register 'p'),Mul 'a' (Value 2),Add 'i' (Value (-1)),Jump (Register 'i') (Value (-2)),Add 'a' (Value (-1)),Set 'i' (Value 127),Set 'p' (Value 622),Mul 'p' (Value 8505),Mod 'p' (Register 'a'),Mul 'p' (Value 129749),Add 'p' (Value 12345),Mod 'p' (Register 'a'),Set 'b' (Register 'p'),Mod 'b' (Value 10000),Send (Register 'b'),Add 'i' (Value (-1)),Jump (Register 'i') (Value (-9)),Jump (Register 'a') (Value 3),Receive 'b',Jump (Register 'b') (Value (-1)),Set 'f' (Value 0),Set 'i' (Value 126),Receive 'a',Receive 'b',Set 'p' (Register 'a'),Mul 'p' (Value (-1)),Add 'p' (Register 'b'),Jump (Register 'p') (Value 4),Send (Register 'a'),Set 'a' (Register 'b'),Jump (Value 1) (Value 3),Send (Register 'b'),Set 'f' (Value 1),Add 'i' (Value (-1)),Jump (Register 'i') (Value (-11)),Send (Register 'a'),Jump (Register 'f') (Value (-16)),Jump (Register 'a') (Value (-19))]
ra = run ins 0 rb
rb = run ins 1 ra

In [9]:
head $ drop 7620 $ scanl (+) 0 $ fmap (const 1) rb

7620